In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/djbetts/opt/anaconda3/lib/

In [3]:
from src.ItemItemRecommender import ItemItemRecommender
from sklearn.model_selection import train_test_split

In [4]:
from datetime import datetime

In [5]:
#Current SQL query
"""
SELECT 
geo__country, 
geo__city,
event_params__video_title,
event_params__videotitle,
event_name,
COUNT(event_name) AS times_completed,
FROM `jfp-data-warehouse.production_firebase.firebase_event` 
WHERE geo__country in ('Russia', 'China', 'Turkey', 'Thailand', 'Iran', 'Vietnam', 'Egypt', 'Japan', 'Bangladesh', 'Pakistan', 'Indonesia', 'India') AND 
event_params__video_title IS NOT NULL AND
event_name IN ('videostarts',  'videoplay',  'a_media_progress10',  'a_media_progress25',  'a_media_progress50',  'a_media_progress75',  'a_media_progress90') AND 
DATE(event_date_partition) 
BETWEEN  "2021-01-01" AND "2021-03-31" 
GROUP BY 
geo__country, 
geo__city,
event_params__videotitle, 
event_params__video_title,
event_name
ORDER BY 
geo__country,
event_params__video_title,
times_completed DESC 
LIMIT 15000
""";

In [6]:
data = pd.read_csv('/Users/djbetts/Desktop/jf_project/data/recommender_base.csv')

In [7]:
df = data.copy()
df.drop(['geo__city', 'times_completed'], axis=1, inplace=True)

In [8]:
df.columns

Index(['geo__country', 'event_params__video_title', 'event_params__videotitle',
       'event_name'],
      dtype='object')

In [9]:
df['video_title'] = df['event_params__videotitle'].fillna(df['event_params__video_title'])

In [10]:
df.drop(['event_params__videotitle','event_params__video_title'], axis=1, inplace=True)

In [11]:
(df.geo__country.value_counts())

India         8644
Egypt         2467
Bangladesh    1953
China         1936
Name: geo__country, dtype: int64

In [12]:
df.event_name.value_counts()

videoplay             2741
videostarts           2670
a_media_progress10    2230
a_media_progress25    2093
a_media_progress50    1908
a_media_progress75    1731
a_media_progress90    1627
Name: event_name, dtype: int64

In [13]:
df.video_title.value_counts()

JESUS                                                   2988
Childhood of Jesus                                       893
Baptism of Jesus by John                                 876
Birth of Jesus                                           856
Abraham                                                  345
                                                        ... 
Jesus Taken Up Into Heaven                                 1
1.우리의 사랑의 추구자이신 예수님                                        1
Magdalena Explains Jesus&#39; Death and Resurrection       1
得救的确据                                                      1
This Is Freedom                                            1
Name: video_title, Length: 406, dtype: int64

In [14]:
df_90 = df[df['event_name'] == 'a_media_progress90']

In [15]:
df_90.video_title.unique()

array(['#FallingPlates', '1. Jesus, Our Loving Pursuer',
       '7. Jesus Our Living Water', 'Angels at the Tomb', 'B Me',
       'Baptism of Jesus by John', 'Beatitudes', 'Birth of Jesus',
       'Blessed are those Who Hear and Obey', 'Breathe',
       'Burial of Jesus', 'Can the Bible Be Trusted?',
       'Childhood of Jesus', 'Creation', 'Crucified Convicts',
       'Death of Jesus', 'Delight', 'Disciples Chosen', 'Doll Face',
       'God&#39;s Rescue Plan', 'Great Commission and Ascension',
       'Healing of the Demoniac', 'Invisible', 'JESUS',
       'Jairus&#39;s Daughter Brought Back to Life',
       'Jesus Calms the Storm', 'Jesus Carries His Cross',
       'Jesus Feeds 5,000', 'Jesus Predicts His Death and Resurrection',
       'Jesus Proclaims Fulfillment of the Scriptures',
       'Jesus Spends Time with Sinners', 'Jesus is Betrayed and Arrested',
       'Jesus is Brought to Herod', 'Jesus is Crucified',
       'Jesus is Mocked and Questioned', 'Jesus is Sentenced',
       

In [16]:
df_90.geo__country.value_counts()

India         937
Egypt         287
Bangladesh    207
China         196
Name: geo__country, dtype: int64

In [17]:
#country_groupby = df.groupby('geo__country')

In [18]:
#country_groupby['video_title'].unique()

In [19]:
#india = country_groupby.get_group('India')

In [20]:
#falling_plates_india = india[india['video_title'] == '#FallingPlates']

In [21]:
#event_score = falling_plates_india.event_name.value_counts()

In [22]:
#event_score.index

In [23]:
#keys = df.video_title.unique()
#reviews = dict.fromkeys(keys, 0)

In [24]:
#tried to make a function by iterating through a groupby. couldn't figure it out.

# def rating_calculator(group):
#     rating = 0
#     reviews = dict.fromkeys(set(group.video_title), 0)
#     for event_name in group.event_name:
#         if event_name == 'videostarts' or event_name == 'videoplay':
#             reviews[group.video_title] += 1
#         elif event_name == 'a_media_progress10':
#             reviews[group.video_title] += (1 * .1)
#         elif event_name == 'a_media_progress25':
#             reviews[group.video_title] += (1 * .25)
#         elif event_name == 'a_media_progress50':
#             reviews[group.video_title] += (1 * .5)
#         elif event_name == 'a_media_progress75':
#             reviews[group.video_title] += (1 * .75)
#         elif event_name == 'a_media_progress90':
#             reviews[group.video_title] += (1 * .9)
#     print(f'total: {rating}\nnormalized: {rating/len(events)}')
#     return (reviews)
# #     return
# # print(rating)
# # print(rating/len(event_score))

In [25]:
#rating_calculator(event_score)

In [26]:
# #broken
# def review_builder(country_groupby):
#     review_dic = dict.fromkeys(country_groupby.groups.keys(), None)
#     print(review_dic)
#     for country, group in country_groupby:
#         review_dic[country] = rating_calculator(group)
#     return review_dic

# function to return score. not normalized

In [27]:
def view_to_rating(event_name):
    d ={
        'videostarts':.01,
        'videoplay':.01, 
        'a_media_progress10':(1*.1),
        'a_media_progress25':(1*.25),
        'a_media_progress50':(1*.5), 
        'a_media_progress75':(1*.75), 
        'a_media_progress90':(1*.9)
    }
    
    return(d[event_name])

In [28]:
view_to_rating('a_media_progress90')

0.9

In [29]:
df['score'] = df.event_name.apply(view_to_rating)

In [30]:
df.drop(['event_name'], axis=1, inplace=True)

In [31]:
df

,geo__country,video_title,score
0,Bangladesh,#FallingPlates,0.01
1,Bangladesh,#FallingPlates,0.10
2,Bangladesh,#FallingPlates,0.90
3,Bangladesh,#FallingPlates,0.01
4,Bangladesh,#FallingPlates,0.75
...,...,...,...
14995,India,JESUS,0.25
14996,India,JESUS,0.50
14997,India,JESUS,0.01
14998,India,JESUS,0.01


In [63]:
groupby = df.groupby(['geo__country','video_title'])

In [64]:
review_scores = groupby.sum()

In [65]:
review_scores

score
geo__country video_title                                
Bangladesh   #FallingPlates                         5.41
             1. Jesus, Our Loving Pursuer           2.52
             2. Jesus, Our Gracious Forgiver        1.62
             6. Jesus, Our Complete Restorer        0.87
             7. Jesus Our Living Water              5.04
...                                                  ...
India        Introduction of Luke                  12.62
             Invisible                             16.46
             Invitation to Know Jesus Personally   37.88
             Isaiah                                40.90
             JESUS                                667.59

[612 rows x 1 columns]

In [35]:
from scipy import sparse
from scipy.sparse import csr_matrix

In [36]:
#df = df.pivot_table(columns='video_title', values='score', index='geo__country', aggfunc='mean', fill_value=0)


In [37]:
df

,geo__country,video_title,score
0,Bangladesh,#FallingPlates,0.01
1,Bangladesh,#FallingPlates,0.10
2,Bangladesh,#FallingPlates,0.90
3,Bangladesh,#FallingPlates,0.01
4,Bangladesh,#FallingPlates,0.75
...,...,...,...
14995,India,JESUS,0.25
14996,India,JESUS,0.50
14997,India,JESUS,0.01
14998,India,JESUS,0.01


In [44]:
number_ratings = (df['score'] > 0).sum()
density = len(review_scores) / (df.shape[0] * df.shape[1])
print('The desnsity is: {} '.format(density))

The desnsity is: 0.0136 


In [48]:
# ratings = (df > 0).sum(axis=1)
# ratings.hist(bins=100)
# print('Min number of ratings: {}'.format(ratings.min()))

In [49]:
# ratings = (df > 0).sum(axis=0)
# ratings.hist(bins=100)
# print('Min number of ratings: {}'.format(ratings.min()))

# rec

In [50]:
df.index

RangeIndex(start=0, stop=15000, step=1)

In [51]:
train, test = train_test_split(df, random_state = 100)

In [58]:
ratings_as_mat = csr_matrix((train.score, 
        ((train.geo__country), (train.video_title))))

ValueError: row, column, and data array must all be the same length

In [59]:
rec = ItemItemRecommender(neighborhood_size=75)
rec.fit(df)

ValueError: could not convert string to float: 'Bangladesh'

In [ ]:
df.T

In [ ]:
rec.ratings_mat['#FallingPlates']

In [ ]:
rec.pred_one_user(user_id='China')

# top rated movies

In [ ]:
average_rating = df.groupby('video_title').mean()['score']

In [ ]:
average_rating.sort_values(ascending=False)

# utility matrix work

In [ ]:
index = df.geo__country.unique()

In [ ]:
columns = df.video_title.unique()

In [ ]:
data = np.zeros((index.shape[0], columns.shape[0]))

In [ ]:
reviews = pd.DataFrame(data, index=index, columns=columns)

In [ ]:
reviews

In [ ]:
df.head()